<a href="https://colab.research.google.com/github/sn715/RandomModel/blob/main/mgenClassical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please don't open it.

Manual: please navigate to "app" section and click on dropdown to select music type. 

In [ ]:
#library for understanding music
from music21 import *

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import *
from collections import Counter
from tqdm import tqdm
import os
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


functions:

In [ ]:
#@title READ MIDI
def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        #if 'Piano' in str(part): 
        
        notes_to_parse = part.recurse() 
  
        #finding whether a particular element is note or a chord
        for element in notes_to_parse:
            
            #note
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            
            #chord
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [ ]:
#@title CREATE FINAL DATA
def createMusicData(musicPath):
  
    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in files])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)            
        new_music.append(temp)
        
    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(100):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    #unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(100)]).reshape(-1))
    #x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))
    #predicted_notes = [x_int_to_note[i] for i in intSong]

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((100, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((100, 32))

    return bigX, bigY

In [ ]:
#@title EMOTION DATA
#CSV DATA

def createEmotionData(emotionData):

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    return final_data


In [1]:
def createData(musicPath, emotionData):

      
    #read all the filenames
    files=[i for i in os.listdir(musicPath) ]

    #reading each midi file
    notes_array = np.array([read_midi(musicPath+i) for i in files])

    #converting 2D array into 1D array
    notes_ = [element for note_ in notes_array for element in note_]

    #No. of unique notes
    unique_notes = list(set(notes_))

    freq = dict(Counter(notes_))
    frequent_notes = [note_ for note_, count in freq.items() if count>=50]

    #concatenating all 100 audio files
    new_music=[]

    for notes in notes_array:
        temp=[]
        for note_ in notes:
            if note_ in frequent_notes:
                temp.append(note_)            
        new_music.append(temp)
        
    new_music = np.array(new_music)

    #cutting all files to 1000
    diff = 0

    for i in range(100):
        diff = len(new_music[i]) - 1000
        if (diff > 0):
          new_music[i] = new_music[i][:-diff]
        elif (diff < 0):
          for j in range(-diff):
            new_music[i].append(0)

    list(set(new_music.ravel()[0]))


    #DICTIONARY unique notes --> numbers
    #unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(100)]).reshape(-1))
    #x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))
    #predicted_notes = [x_int_to_note[i] for i in intSong]

    #preparing input sequences
    x_seq=[]
    for i in new_music:
        temp=[]
        for j in i:
            #assigning unique integer to every note
            temp.append(x_note_to_int[str(j)])
        x_seq.append(temp)

    x_seq = np.array(x_seq)

    x = []
    window_size = 32 #increase
    for i in tqdm(x_seq):
        this_x_ = [] # placeholder
        for j in range(len(i)-window_size):
            this_x_.append(i[j:(j+window_size)])
        x.append(this_x_)

    x_arr = np.asarray(x)
    x_arr_resh = x_arr.reshape((100, 968, 32, 1))

    bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
    bigY = bigY.reshape((100, 32))

    #CSV

    new_data = emotionData.groupby('track id').mean().round(0)
    new_data['track_id'] = new_data.index
    final_data = new_data[['track_id', 'calmness', 'power']]

    return bigX, bigY, final_data

In [ ]:
#@title PREDICT
def predict(music, emotion, model):
    output_of_ONE_SONG = model.predict([music[0:1], emotion[0:1]])
    intSong = list(np.round(output_of_ONE_SONG[0]*100, ).astype(int))

    unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(100)]).reshape(-1))
    x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
    predicted_notes = [x_int_to_note[i] for i in intSong]

    return predicted_notes

In [ ]:
#@title CONVERT TO MIDI
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
                     
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [ ]:
#@title BUILD MODEL
def build_seamise_model_with_functional():
    
    # instantiate the input Tensor
    input_layer = tf.keras.Input(shape=(967, 32, 1))
    input_layer_aux = tf.keras.Input(shape=[3]) # try () or []
    
    # stack the layers using the syntax: new_layer()(previous_layer)
    convlayer = tf.keras.layers.Conv2D(128, (10,2))(input_layer)
    convlayer2 = tf.keras.layers.Conv2D(128, (10,2))(convlayer)
    flatten_layer = tf.keras.layers.Flatten()(convlayer2)
    # flatten_layer_aux = tf.keras.layers.Flatten()(input_layer_aux)
    first_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(flatten_layer)
    second_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(first_dense)
    third_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(second_dense)
    concate = tf.keras.layers.Concatenate()([third_dense, input_layer_aux])
    output_layer = tf.keras.layers.Dense(32)(concate)
    
    # declare inputs and outputs
    # from tensorflow.keras.models import Model
    func_model = tf.keras.models.Model(inputs=[input_layer, input_layer_aux], outputs=output_layer)
    
    return func_model

In [ ]:
#@title CREATE MODEL
#define model
tf.keras.backend.clear_session()
model = build_seamise_model_with_functional()
tf.keras.utils.plot_model(model)
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
opt = tf.keras.optimizers.Adam(
    learning_rate=0.0005,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='adam',
)

model.compile(optimizer=opt, loss='mae') #, metrics=[tf.keras.metrics.Accuracy()])

with tf.device('/device:GPU:0'):
    history = model.fit(
        x=[bigX, final_data],
        y=bigY,
        validation_split = 0.2,
        batch_size=3,
        epochs=7,
        callbacks=[callback])
    
model.save('your_neuralnet_model.h5')

NameError: ignored

In [ ]:
#@title MAGIC FUNCTION
def magic_function():
    if genre == 'classical' and emotion =='relaxation':
        # Recreate the exact same model, including its weights and the optimizer
        new_model = tf.keras.models.load_model('/content/drive/MyDrive/emotifymusicMD/your_neuralnet_model.h5')

    musicPath ="/content/drive/MyDrive/classicalmidi/"

    #/content/drive/MyDrive/emotifymusicMD/emotifyannotationsdata.csv
    csvPath = '/content/drive/MyDrive/emotifymusicMD/'
    os.chdir(csvPath)
    #os.listdir()

    annot = pd.read_csv('emotifyannotationsdata.csv', nrows = 2688)
    annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']
    annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
          'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
          'sadness', 'mood', 'liked', 'disliked']]
    #CSV DATA
    print ("create csv data")
    final_data = createEmotionData(annot_subset)

    print ("create music data")
    bigX, bigY = createMusicData(musicPath)

    print ("finished loading data")

    #CREATE MODEL

    print ("create model")

    tf.keras.backend.clear_session()
    model = build_seamise_model_with_functional()
    tf.keras.utils.plot_model(model)
    keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    opt = tf.keras.optimizers.Adam(
        learning_rate=0.0005,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=False,
        name='adam',
    )

    model.compile(optimizer=opt, loss='mae') #, metrics=[tf.keras.metrics.Accuracy()])

    with tf.device('/device:GPU:0'):
        history = model.fit(
            x=[bigX, final_data],
            y=bigY,
            validation_split = 0.2,
            batch_size=3,
            epochs=7,
            callbacks=[callback])
        
    model.save('/content/drive/MyDrive/emotifymusicMD/mgenModel.h5')

    #PREDICT

    predicted_notes = predict(bigX, final_data, new_model)

    print ("converting output to midi")
    convert_to_midi(predicted_notes)

    !apt install fluidsynth
    !cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
    !fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
    from IPython.display import Audio
    Audio('output.wav')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Default title text
new_model = tf.keras.models.load_model('your_neuralnet_model.h5')

In [ ]:
#@title Default title text
predict(bigX, final_data)

# hide

Music Audio Data

In [ ]:
def read_midi(file):
    
    print("Loading Music File:",file)
    
    notes=[]
    notes_to_parse = None
    
    #parsing a midi file
    midi = converter.parse(file)
  
    #grouping based on different instruments
    s2 = instrument.partitionByInstrument(midi)

    #Looping over all the instruments
    for part in s2.parts:
    
        #select elements of only piano
        #if 'Piano' in str(part): 
        
        notes_to_parse = part.recurse() 
  
        #finding whether a particular element is note or a chord
        for element in notes_to_parse:
            
            #note
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            
            #chord
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    return np.array(notes)

In [ ]:
import os

#Array Processing
import numpy as np

#specify the path
path ="/content/drive/MyDrive/classicalmidi/"


#read all the filenames
files=[i for i in os.listdir(path) ]

#reading each midi file
notes_array = np.array([read_midi(path+i) for i in files])

Loading Music File: /content/drive/MyDrive/classicalmidi/29.mp3.mid
Loading Music File: /content/drive/MyDrive/classicalmidi/7.mp3.mid
Loading Music File: /content/drive/MyDrive/classicalmidi/27.mp3.mid


KeyboardInterrupt: ignored

In [ ]:
#converting 2D array into 1D array
notes_ = [element for note_ in notes_array for element in note_]

#No. of unique notes
unique_notes = list(set(notes_))
print(len(unique_notes))

In [ ]:
#importing library
from collections import Counter

#computing frequency of each note
freq = dict(Counter(notes_))

#library for visualiation
import matplotlib.pyplot as plt

#consider only the frequencies
no=[count for _,count in freq.items()]

#set the figure size
plt.figure(figsize=(5,5))

#plot
plt.hist(no)


In [ ]:
frequent_notes = [note_ for note_, count in freq.items() if count>=50]
print(len(frequent_notes))

In [ ]:
#concatenating all 100 audio files
new_music=[]

for notes in notes_array:
    temp=[]
    for note_ in notes:
        if note_ in frequent_notes:
            temp.append(note_)            
    new_music.append(temp)
    
new_music = np.array(new_music)

In [ ]:
#cutting all files to 1000
diff = 0

for i in range(100):
    diff = len(new_music[i]) - 1000
    if (diff > 0):
      new_music[i] = new_music[i][:-diff]
    elif (diff < 0):
      for j in range(-diff):
        new_music[i].append(0)

In [ ]:
list(set(new_music.ravel()[0]))

In [ ]:
np.unique(np.asarray([new_music[i][0:1000] for i in range(5)]).reshape(-1))

In [ ]:
#DICTIONARY unique notes --> numbers
unique_x = np.unique(np.asarray([new_music[i][0:1001] for i in range(100)]).reshape(-1))
x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [ ]:
x_note_to_int

In [ ]:
#preparing input sequences
x_seq=[]
for i in new_music:
    temp=[]
    for j in i:
        #assigning unique integer to every note
        temp.append(x_note_to_int[str(j)])
    x_seq.append(temp)

x_seq = np.array(x_seq)

In [ ]:
from tqdm import tqdm

In [ ]:
x = []
window_size = 32 #increase
for i in tqdm(x_seq):
    this_x_ = [] # placeholder
    for j in range(len(i)-window_size):
        this_x_.append(i[j:(j+window_size)])
    x.append(this_x_)

In [ ]:
x_arr = np.asarray(x)
x_arr.shape

In [ ]:
x_arr_resh = x_arr.reshape((100, 968, 32, 1))
x_arr_resh.shape

In [ ]:
bigX, bigY = x_arr_resh[:, 0:967, :, :], x_arr_resh[:, 967, :, :]
bigY = bigY.reshape((100, 32))

Emotion Categorical data

In [ ]:
# import
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# directory
path = '/content/drive/MyDrive/emotifymusicMD/'
os.chdir(path)
#os.listdir()

In [ ]:
# annotation data
annot = pd.read_csv('emotifyannotationsdata.csv', nrows = 2688)
annot.shape

(2688, 17)

In [ ]:
annot.columns

In [ ]:
# subset
annot.columns = ['track id', 'genre', 'amazement', 'solemnity', 'tenderness',
       'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
       'sadness', 'mood', 'liked', 'disliked', 'age', 'gender', 'mother tongue']

In [ ]:
# subset only numerical columns
annot_subset = annot[['track id', 'amazement', 'solemnity', 'tenderness',
       'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension',
       'sadness', 'mood', 'liked', 'disliked']]

annot_subset.shape

In [ ]:
# groupby + taking average + rounding to nearest int
new_data = annot_subset.groupby('track id').mean().round(0)
new_data.head(100)
#new_data.shape

In [ ]:
# add track id as a new col
new_data['track_id'] = new_data.index
new_data.head(3)
new_data.shape

In [ ]:
# subset
final_data = new_data[['track_id', 'calmness', 'power']]
final_data

In [ ]:
# save
final_data.to_csv('final_data_for_annotation.csv')

In [ ]:
final_data.shape

# redefine model



In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
def load_music_emotions(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["track_id", "calmness", "power"]
	df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
	
	# return the data frame
	return df

In [ ]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
import keras.backend as K

In [ ]:
#@title PICKLE FILE
import os
import pickle
path = '/content/drive/MyDrive/emotifymusicMD/'
os.chdir(path)
os.listdir()  
a = (bigX, final_data, bigY)

with open('classical_data.pkl', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('classical_data.pkl', 'rb') as handle:
    b = pickle.load(handle)
bigX, final_data, bigY = b 
bigX.shape

Make sure to save your model by using

```
# save model: it must be .h5 format
model.save('your_neuralnet_model.h5')
```

In [ ]:
#@title
import matplotlib.pyplot as plt

In [ ]:
#@title
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('loss')

Assume that your model is trained and saved somewhere. Then when you want to call it, just use the following code

```
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('your_neuralnet_model.h5')
```

In [ ]:
#@title
output_of_ONE_SONG = model.predict([bigX[0:1], final_data[0:1]])
output_of_ONE_SONG.shape

In [ ]:
#@title
intSong = list(np.round(output_of_ONE_SONG[0]*100, ).astype(int))

print(intSong)

load data & train model

In [ ]:
#DO I NEED THIS STEP IT DOESNT WORK LMAOO
"""
#@title
import random
ind = np.random.randint(0,len(bigX)-1)

#random_music = bigX[ind]
no_of_timesteps = 32

predictions=[]
#controls length of piece
for i in range(10):

    #random_music = random_music.reshape(1,no_of_timesteps)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)
"""

In [ ]:
#@title
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
predicted_notes = [x_int_to_note[i] for i in intSong]

In [ ]:
convert_to_midi(predicted_notes)

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
!fluidsynth -ni font.sf2 music.mid -F output.wav -r 44100
from IPython.display import Audio
Audio('output.wav')



```
# This is formatted as code
```



# app

In [ ]:
genre = 'classical' #@param ["classical", "electronic", "pop"]
emotion = 'relaxation' #@param ["relaxation", "energy", "focus"]

In [ ]:
magic_function() # this will have 5a-c inside but it's defined above and hidden so noone will see it.